## Emoji Predictor -- Creating Model To Deploy 

In [1]:
import emoji
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from keras.layers import *
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
pd.set_option('mode.chained_assignment', None)

Using TensorFlow backend.


In [2]:
# print(emoji.EMOJI_UNICODE)
emoji_dictionary = {"0": "\u2764\uFE0F",
                    "1": ":baseball:",
                    "2": ":grinning_face_with_big_eyes:",
                    "3": ":disappointed_face:",
                    "4": ":fork_and_knife:"
                   }

In [3]:
train = pd.read_csv("Dataset/train_emoji.csv", header=None)
test = pd.read_csv("Dataset/test_emoji.csv", header=None)

In [4]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

In [5]:
f = open("glove.6B.50d.txt", encoding='utf-8')

In [6]:
embeddings_index = {}

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float')
    embeddings_index[word] = coefs
    
f.close()

In [7]:
def embedding_output(X):
    maxLen = 10
    emb_dim = 50
    embedding_out = np.zeros((X.shape[0], maxLen, emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for ij in range(len(X[ix])):
            try:
                embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((50,))
    
    return embedding_out

In [8]:
embedding_matrix_train = embedding_output(X_train)
embedding_matrix_test = embedding_output(X_test)

In [9]:
Y_train = to_categorical(Y_train, num_classes=5)
Y_test = to_categorical(Y_test, num_classes=5)

In [10]:
model = Sequential()
model.add(LSTM(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
earlystop = EarlyStopping(monitor="val_accuracy", patience=10)
checkpoint = ModelCheckpoint("Web_Model.h5", monitor='val_loss', verbose=True, save_best_only=True)
hist = model.fit(embedding_matrix_train, Y_train, epochs=100, batch_size=64, shuffle=True, validation_split=0.2, callbacks=[earlystop, checkpoint])

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 2s 17ms/step - loss: 1.6256 - accuracy: 0.1524 - val_loss: 1.6224 - val_accuracy: 0.1852

Epoch 00001: val_loss improved from inf to 1.62236, saving model to Web_Model.h5
Epoch 2/100
105/105 [==============================] - 0s 722us/step - loss: 1.5745 - accuracy: 0.3048 - val_loss: 1.6231 - val_accuracy: 0.2593

Epoch 00002: val_loss did not improve from 1.62236
Epoch 3/100
105/105 [==============================] - 0s 674us/step - loss: 1.5280 - accuracy: 0.3810 - val_loss: 1.6315 - val_accuracy: 0.2593

Epoch 00003: val_loss did not improve from 1.62236
Epoch 4/100
105/105 [==============================] - 0s 665us/step - loss: 1.5297 - accuracy: 0.3333 - val_loss: 1.6467 - val_accuracy: 0.2222

Epoch 00004: val_loss did not improve from 1.62236
Epoch 5/100
105/105 [==============================] - 0s 682us/step - loss: 1.4726 - accuracy: 0.3429 - val_loss: 1.6631 - val_accuracy: 

105/105 [==============================] - 0s 953us/step - loss: 0.2667 - accuracy: 0.9048 - val_loss: 0.9060 - val_accuracy: 0.6296

Epoch 00039: val_loss did not improve from 0.77981
Epoch 40/100
105/105 [==============================] - 0s 889us/step - loss: 0.2644 - accuracy: 0.9143 - val_loss: 0.9630 - val_accuracy: 0.6667

Epoch 00040: val_loss did not improve from 0.77981
Epoch 41/100
105/105 [==============================] - 0s 926us/step - loss: 0.2378 - accuracy: 0.9429 - val_loss: 1.0273 - val_accuracy: 0.6667

Epoch 00041: val_loss did not improve from 0.77981
Epoch 42/100
105/105 [==============================] - 0s 932us/step - loss: 0.1573 - accuracy: 0.9619 - val_loss: 1.0206 - val_accuracy: 0.6667

Epoch 00042: val_loss did not improve from 0.77981


In [12]:
# Loading The Best Model
model.load_weights("Web_Model.h5")

In [13]:
print("Testing Accuracy : ", round(100 * model.evaluate(embedding_matrix_test, Y_test)[1], 2), "%")

56/56 [==============================] - 0s 445us/step
Testing Accuracy :  80.36 %


In [14]:
with open("Web_Model.json", "w") as file:
    file.write(model.to_json())
model.save_weights("Web_Model.h5")

### Code To Be Written In Django Application

In [15]:
from keras.models import model_from_json

In [16]:
with open("Web_Model.json", "r") as file:
    model = model_from_json(file.read())
model.load_weights("Web_Model.h5")

In [17]:
print(model.predict_classes(embedding_matrix_test))

[4 3 2 0 2 2 3 2 4 2 1 2 0 3 1 3 3 2 3 2 3 0 4 3 3 3 2 0 1 2 0 1 3 2 0 2 2
 4 4 2 1 0 0 1 2 0 3 2 3 3 3 0 3 2 2 3]
